In [1]:
import pandas as pd

In [2]:
#UserInfo.tsv
business_by_demography =pd.read_csv('/home/chris/datasets/european_union/test.tsv',delimiter='\t',encoding='utf-8')
print(list(business_by_demography.columns.values)) #file header

['GEO,LEG_FORM,INDIC_SB,NACE_R2\\TIME', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']


In [3]:
print(business_by_demography.head(5)) #last N rows

                  GEO,LEG_FORM,INDIC_SB,NACE_R2\TIME 2006 2007 2008     2009  \
0  European Union (aggregate changing according t...    :    :    :        :   
1  European Union (28 countries),Total,Net busine...    :    :    :        :   
2  European Union (27 countries),Total,Net busine...    :    :    :        :   
3  Belgium,Total,Net business population growth -...    :    :    :  12.26 b   
4  Bulgaria,Total,Net business population growth ...    :    :    :    20.78   

    2010   2011  2012   2013  2014  2015  
0      :      :     :      :     :     :  
1      :      :     :      :     :     :  
2      :      :     :      :     :     :  
3   2.10   2.39  1.33  -0.01  2.29  1.22  
4  -1.15  -1.21  1.19   1.16  1.62  1.92  


## Transpose table and generate new vertical index

In [4]:
#Fill empty cells with zeros
business_by_demography = business_by_demography.fillna(0)
business_by_demography.set_index('GEO,LEG_FORM,INDIC_SB,NACE_R2\TIME',inplace=True)
business_by_demography = business_by_demography.transpose()

## There are certain flags in the table to indicate special features of the data

### Available flags
* b	break in time series
* c	confidential
* d	definition differs, see metadata
* e	estimated
* f	forecast
* i	see metadata (phased out)
* n	not significant
* p	provisional
* r	revised
* s	Eurostat estimate (phased out)
* u	low reliability
* z	not applicable

It is not recommendable in general to drop the information contained by these flags, but for first exploration purposes it should not do too much harm. The flags can easily remove by filling them all into a list and strip them by looping over this list. 

In [5]:
flags = ['b', 'c', 'd','e','f','i','n','p','r','s','u','z']

In [6]:
for column in business_by_demography.columns:  
    column = str(column)
    for flag in flags:
        business_by_demography[column] = business_by_demography[column].str.strip(flag)

## Replace the lengthy name of Germany

In [7]:
for column in business_by_demography.columns:  
    column = str(column)
    column_content = column.split(',')[0]
    if 'Germany' in column_content:
        old_german_name = 'Germany (until 1990 former territory of the FRG)'
        new_german_name = 'Germany'
        new_column = column.replace(old_german_name,new_german_name)
        business_by_demography = business_by_demography.rename(index=str,columns={column:new_column})

## Replace the colons by zeros

In [8]:
business_by_demography = business_by_demography.replace(':',0)

In [9]:
business_by_demography =  business_by_demography.apply(pd.to_numeric, args=('coerce',))
business_by_demography = business_by_demography.fillna(0)

### Drop the years 2006-2008

In [10]:
business_by_demography = business_by_demography.drop('2006').drop('2007').drop('2008')

## Get all unique country names

In [11]:
countries_unique = []
for column in business_by_demography.columns:
    column_content = str(column).split(",")
    countries_unique.append(column_content[0])
countries_unique = list(set(countries_unique))
    

In [12]:
countries_unique

['Sweden',
 'Cyprus',
 'United Kingdom',
 'Spain',
 'European Union (27 countries)',
 'Slovakia',
 'Finland',
 'Latvia',
 'Romania',
 'Turkey',
 'Luxembourg',
 'Netherlands',
 'Malta',
 'Denmark',
 'Poland',
 'Bulgaria',
 'Former Yugoslav Republic of Macedonia',
 'European Union (aggregate changing according to the context)',
 'Lithuania',
 'Italy',
 'Portugal',
 'Slovenia',
 'European Union (28 countries)',
 'Hungary',
 'Austria',
 'France',
 'Czech Republic',
 'Belgium',
 'Switzerland',
 'Germany',
 'Greece',
 'Croatia',
 'Estonia',
 'Norway',
 'Ireland']

In [13]:
import re
df = {}

for country_name in countries_unique:
    df[country_name] = business_by_demography.filter(regex=country_name)

In [14]:
df['Germany']

"GEO,LEG_FORM,INDIC_SB,NACE_R2\TIME","Germany,Total,Net business population growth - percentage,Business economy except activities of holding companies","Germany,Total,Business churn: birth rate + death rate - percentage,Business economy except activities of holding companies","Germany,Total,Death rate: number of enterprise deaths in the reference period (t) divided by the number of enterprises active in t - percentage,Business economy except activities of holding companies","Germany,Total,Employment share of enterprise deaths: number of persons employed in the reference period (t) among enterprise deaths divided by the number of persons employed in t among the stock of active enterprises in t - percentage,Business economy except activities of holding companies","Germany,Sole proprietorship,Net business population growth - percentage,Business economy except activities of holding companies","Germany,Sole proprietorship,Business churn: birth rate + death rate - percentage,Business economy except activities of holding companies","Germany,Sole proprietorship,Death rate: number of enterprise deaths in the reference period (t) divided by the number of enterprises active in t - percentage,Business economy except activities of holding companies","Germany,Sole proprietorship,Employment share of enterprise deaths: number of persons employed in the reference period (t) among enterprise deaths divided by the number of persons employed in t among the stock of active enterprises in t - percentage,Business economy except activities of holding companies","Germany,Partnership, co-operatives, associations, etc.,Net business population growth - percentage,Business economy except activities of holding companies","Germany,Partnership, co-operatives, associations, etc.,Business churn: birth rate + death rate - percentage,Business economy except activities of holding companies","Germany,Partnership, co-operatives, associations, etc.,Death rate: number of enterprise deaths in the reference period (t) divided by the number of enterprises active in t - percentage,Business economy except activities of holding companies","Germany,Partnership, co-operatives, associations, etc.,Employment share of enterprise deaths: number of persons employed in the reference period (t) among enterprise deaths divided by the number of persons employed in t among the stock of active enterprises in t - percentage,Business economy except activities of holding companies","Germany,Limited liability enterprise,Net business population growth - percentage,Business economy except activities of holding companies","Germany,Limited liability enterprise,Business churn: birth rate + death rate - percentage,Business economy except activities of holding companies","Germany,Limited liability enterprise,Death rate: number of enterprise deaths in the reference period (t) divided by the number of enterprises active in t - percentage,Business economy except activities of holding companies","Germany,Limited liability enterprise,Employment share of enterprise deaths: number of persons employed in the reference period (t) among enterprise deaths divided by the number of persons employed in t among the stock of active enterprises in t - percentage,Business economy except activities of holding companies"
2009,-1.18,17.21,8.90,1.39,-2.26,18.79,10.18,4.82,0.16,17.12,7.55,1.30,1.39,12.10,5.79,0.23
2010,0.73,17.28,8.56,1.28,-0.08,18.66,9.66,4.52,2.39,17.83,7.63,1.28,1.99,12.35,5.78,0.16
2011,0.91,17.21,8.55,1.19,1.20,19.17,9.72,4.40,-1.11,16.01,7.52,1.13,1.68,11.90,5.63,0.17
2012,0.41,16.42,8.47,1.14,0.56,18.20,9.74,4.41,-2.46,15.29,7.13,1.03,2.26,11.68,5.55,0.13
2013,-0.85,15.66,8.27,1.14,-0.81,17.07,9.45,4.38,0.46,16.33,7.03,1.14,-1.99,10.64,5.51,0.07
2014,-5.17,15.45,8.25,1.00,-5.48,16.82,9.32,3.53,-12.55,16.56,7.39,1.00,1.77,10.65,5.68,0.08
2015,-0.81,14.74,7.66,0.00,-0.80,16.43,8.88,0.00,-1.11,14.69,6.46,0.00,-0.66,9.79,4.86,0.00


In [15]:
import plotly.plotly as py
import plotly.graph_objs as go

from datetime import datetime
import pandas_datareader.data as web

sole = business_by_demography['Germany,Sole proprietorship,Net business population growth - percentage,Business economy except activities of holding companies']    
total = business_by_demography['Germany,Total,Net business population growth - percentage,Business economy except activities of holding companies']
patnership = business_by_demography['Germany,Partnership, co-operatives, associations, etc.,Net business population growth - percentage,Business economy except activities of holding companies']
limiteds = business_by_demography['Germany,Limited liability enterprise,Net business population growth - percentage,Business economy except activities of holding companies']



sole_proprietorship = go.Scatter(x=df['Germany'].index, y=sole, name='Sole proprietorship')
total_growth = go.Scatter(x=df['Germany'].index, y=total, name='Total growth')
partnerships = go.Scatter(x=df['Germany'].index, y=patnership, name='Partnerships & Associates')
limiteds = go.Scatter(x=df['Germany'].index, y=limiteds, name='Limited liability')

data = [sole_proprietorship,total_growth,partnerships,limiteds]

layout = dict(
    title = "Growth comparison of different legal forms",
    xaxis = dict(
        range = ['2009','2015'])
)

fig = dict(data=data, layout=layout)
py.iplot(fig)

## Look at the total net growth of businesses in the eurozone

In [16]:
import plotly.plotly as py
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')

data = [ dict(
        type = 'choropleth',
        locations = df['CODE'],
        z = df['GDP (BILLIONS)'],
        text = df['COUNTRY'],
        colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '$',
            title = 'GDP<br>Billions US$'),
      ) ]

layout = dict(
    title = '2014 Global GDP<br>Source:\
            <a href="https://www.cia.gov/library/publications/the-world-factbook/fields/2195.html">\
            CIA World Factbook</a>',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='d3-world-map' )

## Build a dataframe to compare the total growth of each country during the period of measurement

### We need to add the country codes to the dataframe for later vizualisation

In [17]:
import pycountry

In [142]:
mapping = {country.name: country.alpha2 for country in pycountry.countries}
codes = []
for column in countries_unique:
    #print(column, mapping.get(column, 'No country found'))
    codes.append((column, mapping.get(column, 'No country found')))
countries_codes = pd.DataFrame(codes, columns=['Country', 'Code'])
#countries_codes = countries_codes.drop(countries_codes[countries_codes.Code == 'No country found'].index)


In [19]:
countries_codes.sort_values(by=['Country'], inplace=True)

## Now it is time to calculate the observable which is to be presented across the map

In [155]:
import numpy as np
import collections #import OrderedDict

mean_growth = []
country_names = []
country_codes = []

mapping = {country.name: country.alpha2 for country in pycountry.countries}

for column in business_by_demography.columns:
    if 'Total,Net business population growth' in column:
        new_column_name = column.split(',')[0]
        #country_names.append(new_column_name)
        mean_growth.append(business_by_demography[column].mean())
        country_codes.append((new_column_name, mapping.get(new_column_name, 'No country found')))

mean_growth_country = pd.DataFrame(np.column_stack([country_codes,mean_growth]), columns=['Country', 'Code', 'Mean growth'])

In [162]:
#mean_growth_country['Country'].drop("European Union*")

In [163]:
import plotly.plotly as py
import plotly.graph_objs as go

data = [go.Bar(
            x = mean_growth_country['Country'],
            y = mean_growth_country['Mean growth']
        
    )]
layout = go.Layout(
    title='January 2013 Sales Report',
)

fig = go.Figure(layout=layout)
py.iplot(data, filename='basic-bar')